Some necessary libraries for langchain


In [1]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "**************************"

Install huggingface_hub


In [3]:
!pip install huggingface_hub

Example 1


In [4]:
from langchain import HuggingFaceHub

In [11]:
llm = HuggingFaceHub(repo_id="google/flan-t5-large" ,model_kwargs={"temperature":0.6, "max_length":64})
llm("Translate English to German: Hi,How are you?")

'Hi, wie Sie?'

Example 2

In [12]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large" ,model_kwargs={"temperature":0.6, "max_length":64})
name = llm.predict("I want to open a restaturant for korean food. Suggest a fancy name for this.")
print(name)

<ipython-input-12-7127cdf59269>:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  name = llm.predict("I want to open a restaturant for korean food. Suggest a fancy name for this.")


Korean fusion


Prompt Templates


In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt_template.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


Chains


In [17]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain

prompt_template = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

llm = HuggingFaceHub(repo_id="google/flan-t5-large" ,model_kwargs={"temperature":0.6, "max_length":64})

llm_chain = LLMChain(prompt=prompt_template, llm=llm)

product = "electric scooters"
name = llm_chain.run(product=product)

print(f"A good name for a company that makes {product} could be: {name}")

<ipython-input-17-6bddba1a81d5>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm)
<ipython-input-17-6bddba1a81d5>:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  name = llm_chain.run(product=product)


A good name for a company that makes electric scooters could be: e-scooters


Types
1. Simple Sequential Chain
2. Sequential Chain

In [6]:
!pip install transformers

In [8]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Define prompts
prompt_1 = PromptTemplate(
    input_variables=["topic"],
    template="Generate three creative and unique marketing slogans for a product related to {topic}."
)

prompt_2 = PromptTemplate(
    input_variables=["slogans"],
    template="Rank the following marketing slogans in order of catchiness and memorability, with 1 being the most catchy: {slogans}"
)

# Load Hugging Face models (consider replacing with suitable models)
generator = pipeline("text-generation", model="facebook/bart-base")  # For slogan generation
ranker = pipeline("text-classification", model="bert-base-uncased")  # For slogan ranking

# Create LLMChains (using pipelines instead of OpenAI)
llm_chain_1 = LLMChain(prompt=prompt_1, llm=generator)
llm_chain_2 = LLMChain(prompt=prompt_2, llm=ranker)

# Create the SequentialChain
sequential_chain = SequentialChain(
    llms=[llm_chain_1, llm_chain_2],
    input_variables=["topic"],
    output_variables=["best_slogan"]
)

# Run the chain
topic = "sustainable energy"
results = sequential_chain.run(topic=topic)

# Extract slogans and rankings
slogans = results["slogans"].split("\n")  # Assuming prompt_1 returns slogans separated by newline
rankings = results["rankings"]

# Find the best slogan based on ranking
best_slogan_index = rankings.index(max(rankings))
best_slogan = slogans[best_slogan_index]

print(f"The best slogan for a product related to {topic} is: {best_slogan}")

Some weights of BartForCausalLM were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...bject at 0x7b4a43b18090>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.pipelines.t...bject at 0x7b4a43b18090>, input_type=TextGenerationPipeline]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of

Agents and tools

In [10]:
!pip install Tool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Created wheel for Tool: filename=tool-0.8.0-py3-none-any.whl size=70550 sha256=10655eb97d851f2e7b5e9666da7b098963d202a0bf07b2dbfce929445e148510
  Stored in directory: /root/.cache/pip/wheels/54/3d/91/22fa3da036d51c305989f94db759559968f8b2c5cc4033a07a
Successfully built Tool


In [11]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent
from langchain.agents.tools import Tool

# Define prompts
translate_prompt = PromptTemplate(
    input_variables=["text"],
    template="Translate the following English text to Spanish: {text}"
)

summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in one sentence: {text}"
)

# Load Hugging Face models
translator = HuggingFaceHub(repo_id="google/flan-t5-base", task="text2text-generation")
summarizer = HuggingFaceHub(repo_id="google/flan-t5-small", task="summarization")

# Create LLMChains for each tool
translate_chain = LLMChain(prompt=translate_prompt, llm=translator)
summarize_chain = LLMChain(prompt=summarize_prompt, llm=summarizer)

# Define tools
tools = [
    Tool(
        name="translator",
        func=translate_chain.run,
        description="Translate English text to Spanish."
    ),
    Tool(
        name="summarizer",
        func=summarize_chain.run,
        description="Summarize a given piece of text."
    )
]

# Load tools and initialize agent
agent = initialize_agent(
    tools=tools,
    llm=HuggingFaceHub(repo_id="google/flan-t5-base"),
    agent="agent:llm-chain",
    verbose=True
)

# User input
user_input = "I would like to translate the sentence 'I love to eat pizza' to Spanish and then summarize it."

# Run the agent
print(agent.run(user_input))

ImportError: cannot import name 'Tool' from 'langchain.agents.tools' (/usr/local/lib/python3.11/dist-packages/langchain/agents/tools.py)

Memory
1. ConversationBufferMemory
2. ConversationChain
3. ConversationBufferWindowMemory

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

# Define the prompt
prompt_template = """
You are a helpful AI assistant.

{history}

User: {user_input}

Assistant:
"""
prompt = PromptTemplate(
    input_variables=["history", "user_input"],
    template=prompt_template
)

# Create an LLM with memory
llm = OpenAI(model_name="text-davinci-003")  # Replace with your preferred model
memory = ConversationBufferMemory()
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# Interact with the chain
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break
    response = llm_chain.run(user_input=user_input)
    print("Assistant:", response)